Task1.3

Result of cProfile

In [1]:
! python -m cProfile -o profile.stats JuliaSet.py

Length of x: 1000
Total elements: 1000000
calculate_z_serial_purepython took 11.474433183670044 seconds


In [2]:
import pstats
p = pstats.Stats("profile.stats")
p.sort_stats("cumulative")
p.print_stats()

Thu Jan 23 22:38:48 2025    profile.stats

         36222023 function calls in 12.166 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000   12.166   12.166 {built-in method builtins.exec}
        1    0.026    0.026   12.166   12.166 JuliaSet.py:1(<module>)
        1    0.492    0.492   12.140   12.140 JuliaSet.py:21(calc_pure_python)
        1    7.373    7.373   11.474   11.474 JuliaSet.py:59(calculate_z_serial_purepython)
 34219980    4.101    0.000    4.101    0.000 {built-in method builtins.abs}
  2002000    0.168    0.000    0.168    0.000 {method 'append' of 'list' objects}
        1    0.007    0.007    0.007    0.007 {built-in method builtins.sum}
        3    0.000    0.000    0.000    0.000 {built-in method builtins.print}
       14    0.000    0.000    0.000    0.000 C:\Program Files\Python39\lib\encodings\cp1252.py:18(encode)
       14    0.000    0.000    0.000    0.000 {built-in meth

We can see that the entry point to the code on line 1 takes a total of 12.16 seconds and ncalls is 1, that means this line executed only once.
Inside calc_pure_python, the call to calculate_z_serial_purepython consumes 11.47 seconds and both functions are called only once. 
Inside calculate_z_serial_purepython, the time spent lines of code (without calling other funktion) is 7.3 seconds. This function makes 34,219,980 calls to abs, which take a total of 4.1, along with other calls that do not cost much time.
The next line i profiled output {method 'append' of 'list' objects}, details the creation of 2,002,000 list items. This creation items is occuring calc_pure_python during the setup phase. Minimal overhead input generation(zs and cs construction).
The final line of the profiling output refers to lsprof. This is the orginal name of the tool that evolved into cProfile and can be ignore.


Result of line_profiler

In [5]:
! python -m kernprof -l JuliaSet_line.py

Length of x: 1000
Total elements: 1000000
calculate_z_serial_purepython took 71.96590089797974 seconds
Wrote profile results to JuliaSet_line.py.lprof
Inspect results with:
C:\Program Files\Python39\python.exe -m line_profiler -rmt "JuliaSet_line.py.lprof"


In [6]:
! python -m line_profiler JuliaSet_line.py.lprof

Timer unit: 1e-06 s

Total time: 73.6534 s
File: JuliaSet_line.py
Function: calc_pure_python at line 21

Line #      Hits         Time  Per Hit   % Time  Line Contents
    21                                           @profile
    22                                           def calc_pure_python(desired_width, max_iterations):
    23                                               """Create a list of complex coordinates (zs) and complex parameters (cs),
    24                                               build Julia set"""
    25         1          1.1      1.1      0.0      x_step = (x2 - x1) / desired_width
    26         1          0.5      0.5      0.0      y_step = (y1 - y2) / desired_width
    27         1          0.3      0.3      0.0      x = []
    28         1          0.4      0.4      0.0      y = []
    29         1          0.3      0.3      0.0      ycoord = y2
    30      1001        382.5      0.4      0.0      while ycoord > y1:
    31      1000        436.2      0.4  

I the line 69 we can see that 41% of the time is spend on the while testing.

Measure Overhead

In [8]:
import time
from JuliaSet import calc_pure_python

if __name__ == "__main__":
    start = time.time()
    ! python JuliaSet.py
    end = time.time()
    print(f"Time without profiler: {end - start} seconds")

Length of x: 1000
Total elements: 1000000
calculate_z_serial_purepython took 6.8797101974487305 seconds
Time without profiler: 7.435228586196899 seconds


In [11]:
import time

if __name__ == "__main__":
    start = time.time()
    ! python -m cProfile -o profile.stats JuliaSet.py
    end = time.time()
    print(f"Time with CProfil: {end - start} seconds")

Length of x: 1000
Total elements: 1000000
calculate_z_serial_purepython took 12.200028896331787 seconds
Time with CProfil: 13.100570917129517 seconds


In [13]:
import time

if __name__ == "__main__":
    start = time.time()
    ! python -m kernprof -l JuliaSet_line.py
    end = time.time()
    print(f"Time with line_Profiler: {end - start} seconds")

Length of x: 1000Time with line_Profiler: 72.221431016922 seconds

Total elements: 1000000
calculate_z_serial_purepython took 69.64492201805115 seconds
Wrote profile results to JuliaSet_line.py.lprof
Inspect results with:
C:\Program Files\Python39\python.exe -m line_profiler -rmt "JuliaSet_line.py.lprof"


Task 1.4

In [22]:
! python -m memory_profiler JuliaSet_line.py

Length of x: 1000
Total elements: 1000000
calculate_z_serial_purepython took 3794.364349603653 seconds
Filename: JuliaSet_line.py

Line #    Mem usage    Increment  Occurrences   Line Contents
    61  121.941 MiB  121.941 MiB           1   @profile
    62                                         def calculate_z_serial_purepython(maxiter, zs, cs):
    63                                             """Calculate output list using Julia update rule"""
    64  129.582 MiB    7.641 MiB           1       output = [0] * len(zs)
    65  132.262 MiB -23229.176 MiB     1000001       for i in range(len(zs)):
    66  132.262 MiB -23229.164 MiB     1000000           n = 0
    67  132.262 MiB -23229.168 MiB     1000000           z = zs[i]
    68  132.262 MiB -23229.176 MiB     1000000           c = cs[i]
    69  132.262 MiB -908201.262 MiB    34219980           while abs(z) < 2 and n < maxiter:
    70  132.262 MiB -884974.910 MiB    33219980               z = z * z + c
    71  132.262 MiB -884975.148 

In [1]:
! python -m mprof run JuliaSet_line.py

Length of x: 1000
Total elements: 1000000
calculate_z_serial_purepython took 6.744645595550537 seconds
mprof.py: Sampling memory every 0.1s
running new process
running as a Python program...


In [2]:
! python -m mprof plot mprofile_20250124152035.dat

Figure(1260x540)


In [3]:
! python -m memory_profiler JuliaSet_line.py

Length of x: 1000
Total elements: 1000000
calculate_z_serial_purepython took 79.87772274017334 seconds
Filename: JuliaSet_line.py

Line #    Mem usage    Increment  Occurrences   Line Contents
    21   44.461 MiB   44.461 MiB           1   @profile
    22                                         def calc_pure_python(desired_width, max_iterations):
    23                                             """Create a list of complex coordinates (zs) and complex parameters (cs),
    24                                             build Julia set"""
    25   44.461 MiB    0.000 MiB           1       x_step = (x2 - x1) / desired_width
    26   44.461 MiB    0.000 MiB           1       y_step = (y1 - y2) / desired_width
    27   44.461 MiB    0.000 MiB           1       x = []
    28   44.461 MiB    0.000 MiB           1       y = []
    29   44.461 MiB    0.000 MiB           1       ycoord = y2
    30   44.539 MiB    0.016 MiB        1001       while ycoord > y1:
    31   44.539 MiB    0.062 MiB   

In [4]:
! python -m mprof run JuliaSet_line.py

Length of x: 1000
Total elements: 1000000
calculate_z_serial_purepython took 6.868818521499634 seconds
mprof.py: Sampling memory every 0.1s
running new process
running as a Python program...


In [5]:
! python -m mprof plot mprofile_20250124154421.dat

Figure(1260x540)


Measure the overhead

In [13]:

import time
import os


start = time.time()
os.system("python JuliaSet_line.py")
end = time.time()
normal_time = end - start
print(f"Time without memory_profiler: {normal_time:.4f} seconds")


start = time.time()
os.system("python -m memory_profiler JuliaSet_line.py")
end = time.time()
profiler_time = end - start
print(f"Time with memory_profiler: {profiler_time:.4f} seconds")


overhead = profiler_time - normal_time
print(f"Memory_profiler overhead: {overhead:.4f} seconds")


Time without memory_profiler: 0.1006 seconds
Time with memory_profiler: 2628.5468 seconds
Memory_profiler overhead: 2628.4462 seconds


In [16]:
import time
import os

start = time.time()
os.system("mprof run JuliaSet_line.py")
end = time.time()
mprof_time = end - start
print(f"Time overhead with mprof: {mprof_time:.4f} second")


Time overhead with mprof: 8.5651 second
